<a href="https://colab.research.google.com/github/SonnetSaif/VAE-from-scratch_PyTorch/blob/main/VAE_PyTorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [2]:
class VariationalAutoEncoder(nn.Module):
  def __init__(self, img_dim, h_dim, z_dim):
    super().__init__()

    self.img_dim_to_h_dim = nn.Linear(img_dim, h_dim)
    self.h_dim_to_mean = nn.Linear(h_dim, z_dim)
    self.h_dim_to_deviation = nn.Linear(h_dim, z_dim)

    self.z_dim_to_hid_dim = nn.Linear(z_dim, h_dim)
    self.hid_dim_to_img_dim = nn.Linear(h_dim, img_dim)


  def encoder(self, x):
    h = nn.Relu(self.img_dim_to_h_dim(x))
    #not sure of loss function for mean and deviation
    mean = self.h_dim_to_mean(h)
    deviation = self.h_dim_to_deviation(h)
    return mean, deviation

  def decoder(self, z):
    z = self.z_dim_to_hid_dim(z)
    # not sure of usage of any loss function
    img = self.hid_dim_to_img_dim(z)
    return img

  def forward(self, x):
    mean, deviation = self.encoder(x)
    z = mean + deviation * (torch.randn_like(deviation))
    x = self.decoder(z)
    return mean, deviation, z